In [1]:
#CREDITS https://jackmckew.dev/sentiment-analysis-text-cleaning-in-python-with-vader.html
#on code for consolidating top sentiments in one video dataset
#then modify code to consolidate top videos with highest overall positive sentiments instead

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import json

analyzer = SentimentIntensityAnalyzer()




In [2]:
video_id = '_O2sg-PGhEg' #video id of df1 in get_sentiment_vader.ipynb 
#text_data = pd.read_table('original_rt_snippets.txt',header=None) 

text_data = pd.read_json(f'../results/crashcourse/{video_id}.json')

In [3]:
def get_sentiment(text:str, analyserobj,desired_type:str='pos'):
    # Get sentiment from text
    sentiment_score = analyserobj.polarity_scores(text)
    return sentiment_score[desired_type]


# Get Sentiment scores
def get_sentiment_scores(df,data_column):
    sentimentlist = [] #empty list should be here because if looped through all videos of a channel, sentimentlist should renew
    
    #df[f'{data_column} negative'] = df[data_column].astype(str).apply(lambda x: get_sentiment(x,analyzer,'neg'))
    df['negative'] = df[data_column].astype(str).apply(lambda x: get_sentiment(x,analyzer,'neg'))
    df['neutral'] = df[data_column].astype(str).apply(lambda x: get_sentiment(x,analyzer,'neu'))
    df['positive'] = df[data_column].astype(str).apply(lambda x: get_sentiment(x,analyzer,'pos'))
    df['compound'] = df[data_column].astype(str).apply(lambda x: get_sentiment(x,analyzer,'compound'))
    
    for comp in df['compound']:
        if comp >= 0.05:
            sentiment = 'positive'
        elif comp <= -0.05:
            sentiment = 'negative'
        else:
            sentiment = 'neutral'
            
        sentimentlist.append(sentiment)

    df['sentiment'] = sentimentlist
    
    return df


In [4]:
text_sentiment = get_sentiment_scores(text_data, 'comment_text')


In [5]:
text_sentiment

,comment_by,comment_text,comment_date,likes_count,negative,neutral,positive,compound,sentiment
0,sam JD,awesome host.. thank u so much,2022-07-14T09:38:00Z,1,0.000,0.377,0.623,0.7650,positive
1,Franzi Z,I love this series more with every episode I w...,2020-06-18T17:26:03Z,4,0.000,0.499,0.501,0.9440,positive
2,Wade Allen,When I see images or renderings of super clust...,2020-06-17T17:46:52Z,0,0.042,0.887,0.071,0.5396,positive
3,Susurrus,It&#39;s kinda like a video game where you try...,2020-06-10T12:41:45Z,1,0.057,0.698,0.245,0.8821,positive
4,Eugene Scott,Can someone shed some light on why a super mas...,2020-06-06T00:10:48Z,0,0.084,0.760,0.156,0.4215,positive
...,...,...,...,...,...,...,...,...,...
1370,Somillian,Under 301 club.,2015-11-05T22:26:29Z,2,0.000,1.000,0.000,0.0000,neutral
1371,Mythri,First Like LOLOL,2015-11-05T22:24:14Z,2,0.000,0.444,0.556,0.3612,positive
1372,Get Fixed,second,2015-11-05T22:23:58Z,0,0.000,1.000,0.000,0.0000,neutral
1373,ZeroKelvin,Second!,2015-11-05T22:23:57Z,0,0.000,1.000,0.000,0.0000,neutral


In [7]:
def print_top_n_reviews(df,data_column,number_of_rows):

    for index,row in df.nlargest(number_of_rows,data_column).iterrows():
        #print(f"Score: {row[data_column]}, Review: {row[0]}")
        print(f"Score: {row[data_column]}, Comment: {row['comment_text']}")
        

    # video_id = '_tULRch1PRQ'
    # if totalpositivesentiment2 > totalnegativesentiment2 and totalneutralsentiment2:
    #     print('df2 video is overall highly positively acclaimed')
    #     print(f'recommend https://www.youtube.com/watch?v={video_id}')

In [8]:
#below are the 5 most positive comments in 1 video
#how to modify functions to print out 5 most positive videos instead

print_top_n_reviews(text_sentiment,'compound', 5)

Score: 0.9889, Comment: Thank you very much, Phil Plait and the others there at <i>Crash Course</i> Astronomy, for making this video! Wow, the Universe really is amazing! I did know about galaxies before watching this video but I did not know about clusters and superclusters. 100 000 000 000 galaxies! Wow, that’s a pretty big number. It’s only a shame that we only will be able visit so few of them...Anyway, quasars are also splendid, fascinating! Thank you for making this video!<br><br>I do have a question, though. You mentioned that there are a number of ways to classify and group galaxies together, but only talked about two grouping them together based on shape (spiral, elliptical, irregular etc.) and based on energy-output (active, non-active and quasars). So galaxies are really huge, but how massive are there? How do you group them together based on mass? How massive are galaxies?
Score: 0.9842, Comment: OK.<br>So I guess I&#39;m gonna rewatch this whole serie from the beginning be

In [9]:
#according to vader github and jack mckew blog, compound scoring is independent, more useful and so it has more meaningful comments and has more depth than positive column contains degrees of sentiments that is dependent on negative and neutral 
print_top_n_reviews(text_sentiment,'positive', 5)

Score: 1.0, Comment: Lol
Score: 1.0, Comment: Brilliant
Score: 1.0, Comment: awesome
Score: 1.0, Comment: Nice
Score: 1.0, Comment: amazing!!!!!!


In [ ]:
#refer to get_top10_sentiment file's loop through all of channel's videos and normalizing every video's comments' unidimensional positives
#then change top10 (by views or likes) dataframe code to top10 (by largest percentages of unidimensional positives based on compound)